In [10]:
import pandas as pd
import random
import os
import ast

# Master Movie Data Manipulation Notebook

How this notebook could work is that we iterate/clean on the 'meta' csv in pandas, and then save it in the repo after we have added the new/corrected/cleaned data

In [11]:
#### change pathname to your clean dataset
meta = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/movies_metadata.csv')
#meta.head(2).T

In [12]:
print(meta.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [13]:
### Adding profit and use of Budget Columns
meta['profit'] = meta['revenue'] - pd.to_numeric(meta['budget'],errors='coerce')
meta['useOfBudget'] = meta['revenue'] / (pd.to_numeric(meta['budget'],errors='coerce') + 000.1)

In [14]:
##Adding production company
#Added Production Companies as a trend
productionCompanies1 = []
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies1.append(ast.literal_eval(meta['production_companies'][rowNum])[0]['name'])
    except:
        productionCompanies1.append('nan')
        
productionCompanies2 = []        
for rowNum in range(meta.shape[0]):
    try: 
        productionCompanies2.append(ast.literal_eval(meta['production_companies'][rowNum])[1]['name'])
    except:
        productionCompanies2.append('nan')
        
meta['ProductionCo1'] = productionCompanies1
meta['ProductionCo2'] = productionCompanies2

In [28]:
#meta[['title','ProductionCo1','ProductionCo2']]

In [15]:
##Adding genre
genre1 = []
for rowNum in range(meta.shape[0]):
    try: 
        genre1.append(ast.literal_eval(meta['genres'][rowNum])[0]['name'])
    except:
        genre1.append('nan')
        
genre2 = []        
for rowNum in range(meta.shape[0]):
    try: 
        genre2.append(ast.literal_eval(meta['genres'][rowNum])[1]['name'])
    except:
        genre2.append('nan')

genre3 = []        
for rowNum in range(meta.shape[0]):
    try: 
        genre3.append(ast.literal_eval(meta['genres'][rowNum])[2]['name'])
    except:
        genre3.append('nan')
meta['genre1'] = genre1
meta['genre2'] = genre2
meta['genre3'] = genre3

## Merging

In [16]:
## Don't run this cell unless you want to merge in ratings (big file)
#Also, update filepath
ratings = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/ratings.csv')
links = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/links.csv')

ratings = ratings.groupby('movieId').agg({'rating':['mean']}).reset_index() #gets average rating
ratings.columns = ['movieId','rating']
linkAvgRating = pd.merge(links,ratings,on='movieId',how='left')

#credits = pd.read_csv('/Users/nickparker/Documents/USF_MSDS/Module_1/593-Visualization/final_project/the-movies-dataset-clean/credits.csv')
#credits.columns = ['cast','crew','movieId']
#linkRatCred = pd.merge(linkAvgRating,credits,on='movieId',how='left')

meta['imdbId'] = pd.to_numeric(meta['imdb_id'].str[2:])
meta = pd.merge(meta,linkAvgRating,on='imdbId',how='left')

In [16]:
cast.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [8]:
meta.head(1).T

,0
adult,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ..."
budget,30000000
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
homepage,http://toystory.disney.com/toy-story
id,862
imdb_id,tt0114709
original_language,en
original_title,Toy Story
overview,"Led by Woody, Andy's toys live happily in his ..."


## Making Flops dataframe

In [32]:
for budget in meta['budget']:
    if type(budget) == str:
        budget = 0

flop = meta.loc[((meta['revenue'] > 0) & pd.to_numeric(meta['budget'],errors='coerce' > 0))]
flop['flop'] = 0
flop.loc[(flop['profit'] < 0),'flop'] = 1

TypeError: '>' not supported between instances of 'str' and 'int'

# Export

#### When you are done with the data manipulation, save as csv in our manipulated data folder

In [9]:
export_csv = meta.to_csv (r'movie_dataset_edited/meta.csv', header=True)
#Don't forget to add '.csv' at the end of the path

In [23]:
export_csv = flop.to_csv (r'movie_dataset_edited/flopDataSet.csv', header=True)
